# Export TensorFlow model

In [ ]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()

In [ ]:
!pip -q install keras

In [ ]:
import os
import keras
import numpy as np
from keras.datasets import fashion_mnist
(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

os.makedirs('./fashion-mnist', exist_ok = True)

np.savez('./fashion-mnist/training', image=x_train, label=y_train)
np.savez('./fashion-mnist/validation', image=x_val, label=y_val)

In [ ]:
prefix = 'tf2-fashion-mnist'

training_input_path   = sess.upload_data('fashion-mnist/training.npz', key_prefix=prefix+'/training')
validation_input_path = sess.upload_data('fashion-mnist/validation.npz', key_prefix=prefix+'/validation')

print(training_input_path)
print(validation_input_path)

### Train on SageMaker

In [ ]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(
    entry_point='fmnist.py',
    source_dir='.',
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    framework_version='2.3.0',
    py_version='py37',
    hyperparameters={'epochs': 10}
)

In [ ]:
tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

### Retrieve model artifact

In [ ]:
tf_estimator.model_data

# On your local machine

## Copy and extract model

```
aws s3 cp <tf_estimator.model_data> .
mkdir tf2-fmnist
tar xf model.tar.gz -C tf2-fmnist
```

## Start Docker
```
docker pull tensorflow/serving
```

```
docker run -p 8501:8501 \
  --mount type=bind,source=/tmp/tfserving/tf2-fmnist,target=/models/tf2-fmnist \
  -e MODEL_NAME=tf2-fmnist -t tensorflow/serving
  
docker stop <container_id>
```
    
    